In [15]:
import pandas as pd
import matplotlib.pyplot as plt


data = pd.read_csv('siren_data_train.csv')
data = data.dropna()

#print(data.head())

#2 - Age and heard
extracted_data = data[['heard', 'age']]

#print(extracted_data.head())

#For every age see how many who heard
hear_by_age = data[data['heard'] == 1].groupby('age').size()

print("Amount that heard the siren by age", hear_by_age)

#For every age see hom many how did not hear
not_hear_by_age = data[data['heard'] == 0].groupby('age').size()

print("Amount that did not hear the siren by age ",not_hear_by_age)



plt.plot(hear_by_age.index, hear_by_age.values, label='Heard')
plt.plot(not_hear_by_age.index, not_hear_by_age.values, label='Did not hear')
plt.xlabel('Age')
plt.ylabel('Amount')
plt.title('Amount that heard/did not hear the siren by age')
plt.legend()
plt.show()

ModuleNotFoundError: No module named 'matplotlib'